# Sanity checks

1. Compare engineered HCRIS PUF file with HCRIS SAS files.

2. Compare engineered HCRIS PUF file with HAC files.
    * Send Tom lines for IL hospitals

3. Recreate penalty amounts

4. Focus on full fiscal years


In [1]:
import pandas as pd
import warnings
from IPython.utils import io
import sys
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
import numpy as np

mydir = '/Users/kenlocey/GitHub/HACRP-HAIs/'

# Part 1: Compare engineered HCRIS PUF file with HCRIS SAS files

### Load engineered HCRIS PUF file

In [2]:
hcris_puf_df = pd.read_pickle('~/GitHub/HACRP-HAIs/data/HCRIS_data/FilteredEngineeredPUF_p5.pkl')
print(hcris_puf_df.shape[0], 'rows, ', hcris_puf_df.shape[1], 'columns')
hcris_puf_df.head()

40703 rows,  43 columns


,RPT_REC_NUM,S2_1_C1_35,S2_1_C1_40,S2_1_C2_40,E_A_HOS_C1_72,E_A_HOS_C1_7096,E_A_HOS_C1_7097,E_A_HOS_C1_7093,E_A_HOS_C1_7090,E_A_HOS_C1_7094,E_A_HOS_C1_7091,E_A_HOS_C1_49,E_A_HOS_C1_50,E_A_HOS_C1_54,E_A_HOS_C1_59,E_A_HOS_C1_68,E_A_HOS_C1_93,E_A_HOS_C1_47,E_A_HOS_C1_48,E_A_HOS_C1_7099,PRVDR_CTRL_TYPE_CD,PRVDR_NUM,NPI,RPT_STUS_CD,FY_BGN_DT,FY_END_DT,PROC_DT,INITL_RPT_SW,LAST_RPT_SW,TRNSMTL_NUM,FI_NUM,ADR_VNDR_CD,FI_CREAT_DT,UTIL_CD,NPR_DT,SPEC_IND,FI_RCPT_DT,FILE_YEAR,Line 19,Reconstructed IPPS payment (pre HAC penalty),Reconstructed HAC penalty,Reconstructed IPPS payment (post HAC penalty),HAC penalty imputed from E_A_HOS_C1_59
2089,699112,1.0,Y,N,76191.0,874.0,14695.0,NaN,NaN,-50.0,NaN,57819.0,4454.0,NaN,62273.0,NaN,NaN,57819.0,5749.0,53.0,2,450219,,2,09/01/2020,12/31/2020,09/29/2021,N,Y,N,04011,4,09/28/2021,F,10/01/2021,,07/20/2021,2020,62273.0,77792.0,778.0,77014.0,622.73
3058,645789,NaN,Y,N,327887.0,NaN,72394.0,NaN,NaN,-53.0,NaN,300956.0,21052.0,NaN,322008.0,NaN,NaN,300956.0,265783.0,59.0,4,370030,,2,09/03/2016,12/31/2016,04/17/2019,N,N,J,04011,4,04/02/2019,F,04/03/2019,,10/12/2017,2016,322008.0,394349.0,3943.0,390406.0,3220.08
3059,725518,NaN,Y,N,5475.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,6,440231,,2,07/01/2018,06/30/2019,09/29/2022,N,N,M,10001,4,09/26/2022,F,09/28/2022,,10/18/2019,2018,NaN,0.0,NaN,0.0,NaN
3060,632292,NaN,N,Y,4965.0,NaN,NaN,NaN,NaN,NaN,NaN,6979.0,563.0,NaN,7542.0,NaN,NaN,6979.0,NaN,75.0,10,490104,,2,07/01/2015,06/30/2016,09/17/2018,N,N,H,11301,4,08/31/2018,F,09/05/2018,,11/29/2016,2015,7542.0,7542.0,75.0,7467.0,75.42
3061,659081,NaN,Y,Y,7048.0,NaN,NaN,NaN,NaN,NaN,NaN,7428.0,608.0,NaN,8036.0,NaN,NaN,7428.0,NaN,80.0,10,490104,,2,07/01/2016,06/30/2017,11/22/2019,N,N,J,11301,4,11/19/2019,F,11/21/2019,,11/27/2017,2016,8036.0,8036.0,80.0,7956.0,80.36


### Load HCRIS (sas7bdat) files as pandas dataframes

In [3]:
with io.capture_output() as captured: df = pd.read_sas('~/GitHub/HCRIS-databuilder/hosp10-sas/prds_hosp10_yr2015.sas7bdat')
df = df.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)
cols = list(df)

hcris_sas_df = pd.DataFrame(columns=cols)

yrs = ['2015', '2016', '2017', '2018', '2019', '2021', '2022']
for yr in yrs:
    
    with io.capture_output() as captured: 
        df = pd.read_sas('~/GitHub/HCRIS-databuilder/hosp10-sas/prds_hosp10_yr'+yr+'.sas7bdat')
    
    df = df.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)
    df['FILE_YEAR'] = [yr] * df.shape[0]
    
    tdf = df[df.duplicated(['prvdr_num'], keep=False)]
    duplicate_hosps = tdf['prvdr_num'].unique().tolist()
    df = df[~df['prvdr_num'].isin(duplicate_hosps)]
    
    hcris_sas_df = pd.concat([hcris_sas_df, df], ignore_index=True)
    

# Rename some columns
to_replace = ['rpt_rec_num', 'prvdr_num', 'fi_num', 'rpt_stus_cd', 'fi_creat_dt', 'fy_bgn_dt', 'fy_end_dt', 
              'util_cd', 'trnsmtl_num', 'state', 'st_cty_cd', 'census', 'region', 'proc_dt', 'msa', 'H3_1_HHA1_C10_15____', 'H3_1_HHA1_C11_15____', 'sub3']

replacement = ['RPT_REC_NUM', 'PRVDR_NUM', 'FI_NUM', 'RPT_STUS_CD', 'FI_CREAT_DT', 'FY_BGN_DT', 'FY_END_DT', 
               'UTIL_CODE', 'TRNSMTL_NUM', 'STATE', 'ST_CTY_CD', 'CENSUS', 'REGION', 'PROC_DT', 'MSA', 'H3_1_HHA1_C10_15','H3_1_HHA1_C11_15', 'SUB3']

hcris_sas_df.rename(columns = {to_replace[0]: replacement[0], to_replace[1]: replacement[1], to_replace[2]: replacement[2], to_replace[3]: replacement[3],
                           to_replace[4]: replacement[4], to_replace[5]: replacement[5], to_replace[6]: replacement[6], to_replace[7]: replacement[7],
                           to_replace[8]: replacement[8], to_replace[9]: replacement[9], to_replace[10]: replacement[10], to_replace[11]: replacement[11],
                           to_replace[12]: replacement[12], to_replace[13]: replacement[13], to_replace[14]: replacement[14], to_replace[15]: replacement[15],
                           to_replace[16]: replacement[16], to_replace[17]: replacement[17]}, inplace = True)

#Convert the report record number (RPT_REC_NUM) to string type
hcris_sas_df['RPT_REC_NUM'] = hcris_sas_df['RPT_REC_NUM'].astype(int).astype(str)
hcris_sas_df.drop(['_NAME_'], axis=1, inplace=True)

print('hcris_sas_df:', hcris_sas_df.shape[0], 'rows,', hcris_sas_df.shape[1], 'columns')
hcris_sas_df.head()


hcris_sas_df: 33495 rows, 5521 columns


RPT_REC_NUM PRVDR_NUM FI_NUM RPT_STUS_CD          FI_CREAT_DT  \
0      571051    660001  01011           1  2016-02-25 00:00:00   
1      572181    341317  11501           1  2016-03-10 00:00:00   
2      572187    421303  11001           1  2016-03-07 00:00:00   
3      572208    421302  11001           1  2016-03-10 00:00:00   
4      572647    511306  11401           1  2016-03-14 00:00:00   

             FY_BGN_DT            FY_END_DT UTIL_CODE TRNSMTL_NUM STATE  \
0  2014-10-01 00:00:00  2015-09-30 00:00:00         F           G   NaN   
1  2014-10-01 00:00:00  2015-09-30 00:00:00         F           G    34   
2  2014-10-01 00:00:00  2015-09-30 00:00:00         F           G    42   
3  2014-10-01 00:00:00  2015-09-30 00:00:00         F           G    42   
4  2014-10-01 00:00:00  2015-09-30 00:00:00         F           G    51   

  ST_CTY_CD CENSUS REGION SUB3              PROC_DT   MSA A_C1_113 A_C1_3  \
0       NaN    NaN    NaN  NaN  2016-02-26 00:00:00   NaN      NaN    NaN   
1     34840    5.0    4.0  NaN  2016-03-16 00:00:00  3120      NaN    NaN   
2     42440    5.0    4.0  NaN  2016-03-16 00:00:00   NaN      NaN    NaN   
3     42190    5.0    4.0  NaN  2016-03-16 00:00:00   NaN      NaN    NaN   
4     51430    5.0    3.0  NaN  2016-03-23 00:00:00   NaN      NaN    NaN   

  A_C1_4201 A_C1_61 A_C1_92 A_C2_4201 A_C2_92 A_C6_4201 A_C6_92 B_1_C0_113  \
0       NaN     NaN     NaN       NaN     NaN       NaN     NaN        NaN   
1       NaN     NaN     NaN       NaN     NaN       NaN     NaN        NaN   
2       NaN     NaN     NaN       NaN     NaN       NaN     NaN        NaN   
3       NaN     NaN     NaN       NaN     NaN       NaN     NaN        NaN   
4       NaN     NaN     NaN       NaN     NaN       NaN     NaN        NaN   

  B_1_C0_114 B_1_C0_3 B_1_C0_4201 B_1_C0_92 B_1_C21_110 B_1_C21_111  \
0        NaN      NaN         NaN       NaN         NaN         NaN   
1        NaN      NaN         NaN       NaN         NaN         NaN   
2        NaN      NaN         NaN       NaN         NaN         NaN   
3        NaN      NaN         NaN       NaN         NaN         NaN   
4        NaN      NaN         NaN       NaN         NaN         NaN   

  B_1_C21_113 B_1_C21_114 B_1_C21_4201 B_1_C21_45 B_1_C21_4501 B_1_C21_61  \
0         NaN         NaN          NaN        NaN          NaN        NaN   
1         NaN         NaN          NaN        NaN          NaN        NaN   
2         NaN         NaN          NaN        NaN          NaN        NaN   
3         NaN         NaN          NaN        NaN          NaN        NaN   
4         NaN         NaN          NaN        NaN          NaN        NaN   

  B_1_C22_110 B_1_C22_111 B_1_C22_113 B_1_C22_114 B_1_C22_4201 B_1_C22_45  \
0         NaN         NaN         NaN         NaN          NaN        NaN   
1         NaN         NaN         NaN         NaN          NaN        NaN   
2         NaN         NaN         NaN         NaN          NaN        NaN   
3         NaN         NaN         NaN         NaN          NaN        NaN   
4         NaN         NaN         NaN         NaN          NaN        NaN   

  B_1_C22_4501 B_1_C22_61 B_1_C26_113 B_1_C26_114 B_1_C26_200 B_1_C26_4201  \
0          NaN        NaN         NaN         NaN         NaN          NaN   
1          NaN        NaN         NaN         NaN         NaN          NaN   
2          NaN        NaN         NaN         NaN         NaN          NaN   
3          NaN        NaN         NaN         NaN         NaN          NaN   
4          NaN        NaN         NaN         NaN         NaN          NaN   

  B_2_C26_113 B_2_C26_114 B_2_C26_4201 B_2_C26_61 C_1_C5_113 C_1_C5_114  \
0         NaN         NaN          NaN        NaN        NaN        NaN   
1         NaN         NaN          NaN        NaN        NaN        NaN   
2         NaN         NaN          NaN        NaN        NaN        NaN   
3         NaN         NaN          NaN        NaN        NaN        NaN   
4         NaN      

## Comparisons between SAS-based HCRIS files and engineered PUF HCRIS files

In [4]:
cols = list(hcris_puf_df)
hcris_sas_df = hcris_sas_df.filter(items=cols, axis=1)

print('The engineered HCRIS PUF file contains', hcris_puf_df.shape[1], 'features')
print('The SAS based HCRIS file contains', hcris_sas_df.shape[1], 'of these features')

The engineered HCRIS PUF file contains 43 features
The SAS based HCRIS file contains 27 of these features


In [5]:
yrs = ['2015', '2016', '2017', '2018', '2019', '2021', '2022']
for yr in yrs:
    
    puf = hcris_puf_df[hcris_puf_df['FILE_YEAR'] == yr]
    sas = hcris_sas_df[hcris_sas_df['FILE_YEAR'] == yr]
    
    p1 = len(puf['PRVDR_NUM'].unique())
    p2 = len(sas['PRVDR_NUM'].unique())
    print('File year:', yr, '|', p1, 'hospitals in the PUF file, ', p2, 'hospitals in the SAS file')
    

File year: 2015 | 6124 hospitals in the PUF file,  6001 hospitals in the SAS file
File year: 2016 | 6107 hospitals in the PUF file,  6008 hospitals in the SAS file
File year: 2017 | 6076 hospitals in the PUF file,  5980 hospitals in the SAS file
File year: 2018 | 6048 hospitals in the PUF file,  5942 hospitals in the SAS file
File year: 2019 | 6013 hospitals in the PUF file,  5912 hospitals in the SAS file
File year: 2021 | 3671 hospitals in the PUF file,  3634 hospitals in the SAS file
File year: 2022 | 17 hospitals in the PUF file,  18 hospitals in the SAS file


In [6]:
# Convert select date columns to a common format (Y-M-D)

puf_df = hcris_puf_df.copy(deep=True)
sas_df = hcris_sas_df.copy(deep=True)

date_cols = ['FY_BGN_DT', 'FY_END_DT', 'PROC_DT', 'FI_CREAT_DT', 'NPR_DT', 'FI_RCPT_DT']

for c in date_cols:
    
    if c in list(sas_df):
        sas_df[c] = pd.to_datetime(sas_df[c])
        sas_df.style.format({c: lambda t: t.strftime("%Y-%m-%d")})
    
    if c in list(puf_df):
        puf_df[c] = pd.to_datetime(puf_df[c])
        puf_df.style.format({c: lambda t: t.strftime("%Y-%m-%d")})
    
    

## Compare the SAS-based and engineered HCRIS files, year by year and column by column

In [7]:
yrs = ['2015', '2016', '2017', '2018', '2019', '2021', '2022']
for yr in yrs:
    
    print('Year:', yr)
    puf = puf_df[puf_df['FILE_YEAR'] == yr]
    sas = sas_df[sas_df['FILE_YEAR'] == yr]
    
    # get hospitals contained in both files
    p1 = puf['PRVDR_NUM'].unique()
    p2 = sas['PRVDR_NUM'].unique()
    p3 = list(set(p1) & set(p2))    
    puf = puf[puf['PRVDR_NUM'].isin(p3)]
    sas = sas[sas['PRVDR_NUM'].isin(p3)]
    
    # get columns contained in both files
    cols = list(sas_df)
    
    n_100 = 0
    n_99 = 0
    n_95 = 0
    
    for c in cols:
        pc = 0
        #print('  --  Feature:', c)
        
        sas[c].fillna(0, inplace=True)
        puf[c].fillna(0, inplace=True)
        
        sas.sort_values(by=['PRVDR_NUM'], ascending=True, inplace=True)
        puf.sort_values(by=['PRVDR_NUM'], ascending=True, inplace=True)
        
        sas_c = sas[c].tolist()
        puf_c = puf[c].tolist()
        
        if len(sas_c) != len(puf_c):
            print('      len(sas_c) != len(puf_c)')
            print('      len(sas_c):', len(sas_c))
            print('      len(puf_c):', len(puf_c), '\n')
            break
        
        elif len(sas_c) == len(puf_c):
            
            for i, v1 in enumerate(sas_c):
                v2 = puf_c[i]
                if v1 == v2:
                    pc += 1
                else:
                    pass
                
            pc = np.round(100*pc/len(sas_c),2)        
            
            if pc == 100:
                n_100 += 1
            elif pc > 99:
                print(c, 'is', pc, '% identical between databases')
                n_99 += 1
            elif pc > 95:
                n_95 += 1
                
    if n_99 > 0:
        print('No. of columns that are more than 99% but less than 100% identical:', n_99)
        print('No. of columns that are 100% identical:', n_100, '\n')
    else:
        print('No. of columns that are 100% identical:', n_100, '\n')


Year: 2015
E_A_HOS_C1_7099 is 99.98 % identical between databases
No. of columns that are more than 99% but less than 100% identical: 1
No. of columns that are 100% identical: 26 

Year: 2016
No. of columns that are 100% identical: 27 

Year: 2017
No. of columns that are 100% identical: 27 

Year: 2018
No. of columns that are 100% identical: 27 

Year: 2019
E_A_HOS_C1_7096 is 99.97 % identical between databases
E_A_HOS_C1_7097 is 99.97 % identical between databases
E_A_HOS_C1_7093 is 99.97 % identical between databases
E_A_HOS_C1_7094 is 99.97 % identical between databases
E_A_HOS_C1_49 is 99.97 % identical between databases
E_A_HOS_C1_50 is 99.97 % identical between databases
E_A_HOS_C1_54 is 99.98 % identical between databases
E_A_HOS_C1_59 is 99.97 % identical between databases
E_A_HOS_C1_47 is 99.97 % identical between databases
E_A_HOS_C1_48 is 99.97 % identical between databases
E_A_HOS_C1_7099 is 99.95 % identical between databases
No. of columns that are more than 99% but less 

------

# Part 2: Compare HAC files with the engineered HCRIS file

### Load, process, format HAC file

In [8]:
#############################################################################################
########################   HAC DATA   #######################################################
#############################################################################################

hac_df = pd.read_pickle(mydir + "data/CareCompare_data/CombinedFiles_HACRP/Facility.pkl")
print('hac_df.shape:', hac_df.shape)

######################## 2015 (add penalty assignments) #####################################

df_2015 = hac_df[hac_df['file_year'] == '2015']
df_2015['Total HAC Score'] = pd.to_numeric(df_2015['Total HAC Score'], errors='coerce')
df_2015_new = pd.DataFrame(columns=list(df_2015))
mos = ['01', '04', '05', '07', '10'] # 12 is for FY 2016
for mo in mos:
    
    tdf = df_2015[df_2015['file_month'] == mo]
    hacs = tdf['Total HAC Score'].tolist()
    
    pr = []
    m = np.nanpercentile(hacs, 75, axis=0)
    
    for i in hacs:
        if i >= m:
            pr.append('Yes')
        elif i < m:
            pr.append('No')
        else:
            pr.append('No')
            
    tdf['Payment Reduction'] = pr
    
    df_2015_new = pd.concat([df_2015_new, tdf], axis=0)

hac_df = hac_df[hac_df['file_year'] != '2015']
hac_df = pd.concat([hac_df, df_2015_new], axis=0)
    

######################## 2016 (add penalty assignments) ####################################$

df_2016 = hac_df[hac_df['file_year'] == '2016']
df_2016_new = pd.DataFrame(columns=list(df_2016))
mos = ['05', '08', '11'] # 12 is for FY 2017

for mo in mos:
    
    tdf = df_2016[df_2016['file_month'] == mo]
    hacs = tdf['Total HAC Score'].tolist()
    
    pr = []
    m = np.nanpercentile(hacs, 75, axis=0)
    
    for i in hacs:
        if i >= m:
            pr.append('Yes')
        elif i < m:
            pr.append('No')
        else:
            pr.append('No')
            
    tdf['Payment Reduction'] = pr
    
    df_2016_new = pd.concat([df_2016_new, tdf], axis=0)

hac_df = hac_df[hac_df['file_year'] != '2016']
hac_df = pd.concat([hac_df, df_2016_new], axis=0)
#print('hac_df.shape:', hac_df.shape)


########################  format HAC dates  #################################################

## Fix Nov 2016 date format, convert to default datetime
tdf1 = hac_df[(hac_df['file_year'] == '2016') & (hac_df['file_month'] == '11')]
tdf1['Domain 2 Start Date'] = '0' + tdf1['Domain 2 Start Date'].astype(str)
tdf1['Domain 2 Start Date'] = pd.to_datetime(tdf1['Domain 2 Start Date'], format='%m%d%Y')
tdf1.style.format({'Domain 2 Start Date': lambda t: t.strftime("%Y-%m-%d")})
tdf1['Domain 2 End Date'] = pd.to_datetime(tdf1['Domain 2 End Date'], format='%m%d%Y')
tdf1.style.format({'Domain 2 End Date': lambda t: t.strftime("%Y-%m-%d")})
hac_df['year_month'] = hac_df['file_year'] + hac_df['file_month']
hac_df = hac_df[hac_df['year_month'] != '201611']
hac_df.drop(labels='year_month', inplace=True, axis=1)

## Fix Nov 2017 date format, convert to default datetime
tdf = hac_df[(hac_df['file_year'] == '2017') & (hac_df['file_month'] == '10')]
tdf['Domain 2 Start Date'] = '0' + tdf['Domain 2 Start Date'].astype(str)
tdf['Domain 2 Start Date'] = pd.to_datetime(tdf['Domain 2 Start Date'], format='%m%d%Y')
tdf.style.format({'Domain 2 Start Date': lambda t: t.strftime("%Y-%m-%d")})
tdf['Domain 2 End Date'] = pd.to_datetime(tdf['Domain 2 End Date'], format='%m%d%Y')
tdf.style.format({'Domain 2 End Date': lambda t: t.strftime("%Y-%m-%d")})
hac_df['year_month'] = hac_df['file_year'] + hac_df['file_month']
hac_df = hac_df[hac_df['year_month'] != '201710']
hac_df.drop(labels='year_month', inplace=True, axis=1)

## Convert all other HAC file data to default datetime
hac_df['Domain 2 End Date'] = pd.to_datetime(hac_df['Domain 2 End Date'])
hac_df.style.format({'Domain 2 End Date': lambda t: t.strftime("%Y-%m-%d")})

hac_df['HAI Measures Start Date'] = pd.to_datetime(hac_df['HAI Measures Start Date'])
hac_df.style.format({'HAI Measures Start Date': lambda t: t.strftime("%Y-%m-%d")})

hac_df['HAI Measures End Date'] = pd.to_datetime(hac_df['HAI Measures End Date'])
hac_df.style.format({'HAI Measures End Date': lambda t: t.strftime("%Y-%m-%d")})

print('rows in hac_df, after reformatting dates:', hac_df.shape[0] + tdf1.shape[0] + tdf.shape[0])
hac_df = pd.concat([hac_df, tdf1, tdf], axis=0)
hac_df.sort_values(by='HAI Measures End Date', inplace=True)

hac_df = hac_df.filter(items=['Facility ID', 'file_month', 'file_year', 'Payment Reduction'])
hac_df = hac_df[((hac_df['file_month'] == '10') & (hac_df['file_year'] != '2016')) | ((hac_df['file_month'] == '11') & (hac_df['file_year'] == '2016')) | ((hac_df['file_month'] == '04') & (hac_df['file_year'] == '2022'))]


hac_df.shape: (108489, 44)
rows in hac_df, after reformatting dates: 101818


### Define a function to compare HAC and HCRIS files

In [11]:
def get_intrxns(hacrp_df, hcris_df, year):
    
    hacrp_df = hacrp_df[hacrp_df['file_year'] == year]
    hcris_df = hcris_df[hcris_df['FILE_YEAR'] == year]
    
    hcris_hosps = hcris_df['PRVDR_NUM'].unique().tolist()
    print('   ', len(hcris_hosps), 'hospitals in the HCRIS data')
    
    # 'Imputed HAC penalty amount'
    # 'E_A_HOS_C1_7099'
    hcris_pen_hosps = hcris_df[hcris_df['Reconstructed HAC penalty'] > 0]['PRVDR_NUM'].unique().tolist()
    print('   ', '     ...', len(hcris_pen_hosps), 'have calculated amounts for HAC penalties')
    
    hac_hosps = hacrp_df['Facility ID'].unique().tolist()
    print('   ', len(hac_hosps), 'hospitals in the HACRP data')
    
    hac_pen_hosps = hacrp_df[hacrp_df['Payment Reduction'] == 'Yes']['Facility ID'].unique().tolist() # focus on penalized hospitals
    print('   ', '     ...', len(hac_pen_hosps), 'were assigned penalties')
    
    
    inHACRP_notinHCRIS = len(np.setdiff1d(hac_pen_hosps, hcris_pen_hosps).tolist())
    print('   ', inHACRP_notinHCRIS, "hospitals with penalty assignments (HAC data) do not have penalty amounts (HCRIS data)")
    
    inHCRIS_notinHACRP = len(np.setdiff1d(hcris_pen_hosps, hac_pen_hosps).tolist())
    print('   ', inHCRIS_notinHACRP, "hospitals with penalty amounts (HCRIS data) do not have penalty assignments (HAC data)")
    
    
    #if inHCRIS_notinHACRP > 0:
    #    print('   ', inHCRIS_notinHACRP, 'hospitals with penalty amounts in the HCRIS data are not in the HACRP data.')
    #elif inHCRIS_notinHACRP == 0:
    #    print('   ', 'However, all hospitals with penalty amounts in the HCRIS data are in the HACRP data.')
    
    #inHCRIS_notinHACRP = len(np.setdiff1d(hcris_hosps, hac_pen_hosps).tolist())
    #if inHCRIS_notinHACRP > 0:
    #    print('   ', inHCRIS_notinHACRP, 'hospitals with penalty amounts in the HCRIS file do not have penalty assignments in the HACRP file.\n')
    
    print('\n')
    return

## Compare HAC and HCRIS files

In [12]:
yrs = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

for yr in yrs:

    print('Year:', yr)
    get_intrxns(hac_df, puf_df, yr)
    
    

Year: 2015
    6124 hospitals in the HCRIS data
         ... 3381 have calculated amounts for HAC penalties
    3336 hospitals in the HACRP data
         ... 847 were assigned penalties
    16 hospitals with penalty assignments (HAC data) do not have penalty amounts (HCRIS data)
    2550 hospitals with penalty amounts (HCRIS data) do not have penalty assignments (HAC data)


Year: 2016
    6107 hospitals in the HCRIS data
         ... 3341 have calculated amounts for HAC penalties
    3352 hospitals in the HACRP data
         ... 891 were assigned penalties
    15 hospitals with penalty assignments (HAC data) do not have penalty amounts (HCRIS data)
    2465 hospitals with penalty amounts (HCRIS data) do not have penalty assignments (HAC data)


Year: 2017
    6076 hospitals in the HCRIS data
         ... 3316 have calculated amounts for HAC penalties
    3314 hospitals in the HACRP data
         ... 768 were assigned penalties
    19 hospitals with penalty assignments (HAC data) do no

### Conclusion

HCRIS data often contain HAC reduction adjustment amounts (E_A_HOS_C1_7099) for hospitals that weren't actually penalized by the HACRP or that weren't even included in the HACRP. This suggests that E_A_HOS_C1_7099 is often calculated as a matter of course.
      
      
      